In [240]:
import os

import pandas as pd
import numpy as np
import re


from unidecode import unidecode #para los acentos

In [241]:
def export_to_csv(data_frame, file_name):
    """
    Export a DataFrame to a CSV file.

    Parameters:
        - data_frame: pandas DataFrame
        - file_name: str, name of the CSV file (without the extension)
    """
    #print date stamp - datetime.datetime.now()
    
    #t_stamp = str(datetime.datetime.now()).split(".")[0].replace(" ","_")
    
    file_path = f"{file_name}.csv"
    
    data_frame.to_csv(file_path, index=False)
    print(f"DataFrame successfully exported to {file_path}")

## Empieza aqui:

In [216]:
df = pd.read_csv("../../data/mercadona-scrape-2023/Mercadona Scraping 2023-03-11_21-39-33.csv", delimiter = "~")

In [217]:
# Seleccionamos las columnas que nos interesan: 
df = df[["product", "product_type","product_volume", "product_code", "product_price","collected_timestamp"]]

In [218]:
df.sample()

,product,product_type,product_volume,product_code,product_price,collected_timestamp
5091,Yogur griego natural Hacendado,Not available,Not available,20559.0,1.45,2023-03-12 09:43:14.456699


In [219]:
## Aplicamos las tranformaciones pertinentes para ajustar este formato al nuestro (Srape-2024)

In [220]:
# Si contiene este caracter "|" --> lo vamos a dejar como np.nan

def uniform(x):
    
    if "|" in x:
        return np.nan
    else:
        return x

In [221]:
df["collected_timestamp"] = df["collected_timestamp"].str[0:11]

# eliminamos todas las celdas donde esta descuadrado el precio por unidad -- yo no lo saque
df["product_volume"] = df["product_volume"].apply(uniform)
# Sustituimos los np.nan/not available por cadena vacia -- sino luego al concatenar nos da errores
df['product_volume'].fillna('', inplace=True)
df['product_volume'] = df['product_volume'].str.replace("Not available", "")
df['product_type'] = df['product_type'].str.replace("Not available", "")
df['unidades'] = df['product_type'].str.cat(df['product_volume'], sep=' ')

In [228]:
df = df[["product", "unidades", "product_code", "product_price", "collected_timestamp"]]

df.rename(columns={ "product": "descripcion", "product_price" : "price_eu"}, inplace = True)

## DUPLICADOS

In [229]:
def dupli_rows(df):
    "How many duplicate rows there is"
    duplicates = df.shape[0] - df.drop_duplicates().shape[0]
    print(f'There is {duplicates} duplicate rows')
    return 

dupli_rows(df)

There is 0 duplicate rows


In [230]:
df.drop_duplicates(inplace=True)

In [231]:
df.shape

(4973, 5)

## MARCA

In [232]:
"""Funcion que te devuelve la marca si el texto la contiene"""
def marca(x):
    #marcas = ["Hacendado", "Deliplus", "Bosque Verde", "Coca-Cola"]
    
    marcas = ["Hacendado", "Deliplus", "Bosque Verde", "Coca-Cola", "Aquarius", "Compy", "L'Oréal", "Delikuit", 
                "Maybelline", "Gillette", "Nivea", "Puleva", "Incarlopsa", "Tampax", "Gallina Blanca", 
                "El Pozo", "Hero baby", "Finish", "Garnier", "Rimmel London", "Pantene", "Evax", 
                "Dolce Gusto", "Entrepinares", "Mahou"]
    
    
    return [i for i in marcas if i in x]

In [233]:
#Aplicamos la funcion, como realmente devuelve una lista, si esta vacia (no match) --> np.nan
df["marca"] = df["descripcion"].apply(marca).apply(lambda x: x[0] if x != [] else np.nan)

In [234]:
df

,descripcion,unidades,product_code,price_eu,collected_timestamp,marca
0,"Aceite de oliva 0,4º Hacendado",Garrafa 5 L,4241.0,23.63,2023-03-11,Hacendado
1,"Aceite de oliva 0,4º Hacendado",Botella 1 L,4240.0,4.77,2023-03-11,Hacendado
2,Aceite de oliva virgen extra Hacendado,Garrafa 3 L,4717.0,16.75,2023-03-11,Hacendado
3,Aceite de oliva virgen extra Hacendado,Botella 1 L,4740.0,5.63,2023-03-11,Hacendado
4,Aceite de oliva virgen extra Hacendado Gran Se...,Botella 750 ml,4706.0,5.11,2023-03-11,Hacendado
...,...,...,...,...,...,...
5224,Bebida de pomelo Hacendado sin azúcares añadidos,,39604.0,1.25,2023-03-12,Hacendado
5225,Bebida guayaba Hacendado sin azúcares añadidos,,39659.0,1.30,2023-03-12,Hacendado
5226,Bebida de pera Hacendado sin azúcares añadidos,,39628.0,1.10,2023-03-12,Hacendado
5227,Zumo de pomelo Hacendado,,39696.0,1.45,2023-03-12,Hacendado


In [242]:
def format_units(i):
    
    # Format Ex:         (contains "escurrido")
    pattern_1 = r'^(\S+)\s+(\d+(?:,\d+)?\s*[kKgGmlML]+)\s+(\d+\s*[kKgGmlML]+)\s*escurrido$'

    # Fromat Ex:
    pattern_2 = r'^([A-Z][a-zA-Z]+)\s((?:\d+,)?\d+\s(?:[Kk]g|g|L|ml))'

    #Formaat Ex: '8 latas x 330 ml
    pattern_3 = r'(\d+)\s([a-zA-Z\s]+)\sx\s((?:\d+,)?\d+\s(?:kg|ml|g|L))'
    
    #Format Ex:
    pattern_4 = r'(?:\d+,)?\d+\s+(?:kg|g|ml|L)'
    
    
    if re.match(pattern_1, i):

        matches = re.findall(pattern_1, i)

        formato = matches[0][0]  #.capitalize() ya veremos 

        cantidad = f'1 x {matches[0][2]}'

        #Funciona bien -- para comprobar:
        #print(f'{i} ---- {formato} / {cantidad}')

        return [formato, cantidad]


    # Formato 1: Paquete 500 g / Tarro 570 g (400 g escurrido)
    if re.match(pattern_2, i):

        matches = re.findall(pattern_2, i)

        formato = matches[0][0]  #.capitalize() ya veremos 

        cantidad = f'1 x {matches[0][1]}'

        #Funciona bien -- para comprobar:
        #print(f'{i} ---- {formato} / {cantidad}')

        return [formato, cantidad]


    #Formato 2: '8 latas x 330 ml
    if re.match(pattern_3, i):

        matches = re.findall(pattern_3, i)

        cantidad = f'{matches[0][0]} x {matches[0][2]}'
        formato = matches[0][1]  #.capitalize() ya veremos 

        #Funciona bien -- para comprobar:
        #print(f'{i} ---- {formato} / {cantidad}')

        return [formato, cantidad]

    if re.search(pattern_4, i):

        matches = re.findall(pattern_4, i)

        cantidad = f'1 x {matches[0]}'

        formato = i.replace(matches[0], "")

        #print(f'{i} ---- {formato} / {cantidad}')

        return [formato, cantidad]

    else:
        formato = i
        cantidad = np.nan
        #print(f'{i} ---- {formato} / {cantidad}')

        return [formato, cantidad]




In [243]:
# Por el tema de acentos
df["unidades"] = df["unidades"].apply(unidecode)

#con esto eliminos los puntos de ud. y de aprox.
df["unidades"] = df["unidades"].str.replace(".", "")

# me quito todods los aprox
df["unidades"] = df["unidades"].str.replace("aprox", "")

df["unidades"] = df["unidades"].str.replace("(", "")

df["unidades"] = df["unidades"].str.replace(")", "")

df["format"] = df["unidades"].apply(format_units).apply(lambda x:x[0])
df["quantity"] = df["unidades"].apply(format_units).apply(lambda x:x[1])


df.drop("unidades", axis=1, inplace=True)

In [244]:
df

,descripcion,product_code,price_eu,collected_timestamp,marca,format,quantity
0,"Aceite de oliva 0,4º Hacendado",4241.0,23.63,2023-03-11,Hacendado,Garrafa,1 x 5 L
1,"Aceite de oliva 0,4º Hacendado",4240.0,4.77,2023-03-11,Hacendado,Botella,1 x 1 L
2,Aceite de oliva virgen extra Hacendado,4717.0,16.75,2023-03-11,Hacendado,Garrafa,1 x 3 L
3,Aceite de oliva virgen extra Hacendado,4740.0,5.63,2023-03-11,Hacendado,Botella,1 x 1 L
4,Aceite de oliva virgen extra Hacendado Gran Se...,4706.0,5.11,2023-03-11,Hacendado,Botella,1 x 750 ml
...,...,...,...,...,...,...,...
5224,Bebida de pomelo Hacendado sin azúcares añadidos,39604.0,1.25,2023-03-12,Hacendado,,NaN
5225,Bebida guayaba Hacendado sin azúcares añadidos,39659.0,1.30,2023-03-12,Hacendado,,NaN
5226,Bebida de pera Hacendado sin azúcares añadidos,39628.0,1.10,2023-03-12,Hacendado,,NaN
5227,Zumo de pomelo Hacendado,39696.0,1.45,2023-03-12,Hacendado,,NaN


In [ ]:
### EXPORTAR - OJO NO ESTAN LAS CATEGORIAS SUBCATEGORIAS: 

In [245]:
# Exportar descripciones y códigos: